In [1]:
# import necessary modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

#files
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
#import dataset for modeling
df = pd.read_csv(r'C:/Users/foeke/OneDrive/Documenten/submitting paper/All scripts - paper/data/GlobalModelData/PredictingDataset.csv', sep=',')
df

,Unnamed: 0,Longitude,Latitude,mean_value_NO2,nightlight_450,nightlight_3150,population_1000,population_3000,road_class_2_25,road_class_3_3000,road_class_3_300,trop_mean_filt_2019,BldDen100,NDVI,trafBuf25,trafBuf50
0,0,10.008293,48.397079,29.250500,22.585377,13.402575,8123.319336,54316.64453,0.000000,203502.42190,4167.064941,0.000058,1.000663,2310.0,0.000000,0.000000
1,1,10.021227,53.523262,32.098936,23.546194,22.862799,9558.207031,76786.28125,0.000000,99596.04688,1850.278076,0.000081,0.978245,2518.0,0.000000,0.000000
2,2,10.053859,53.592301,58.703744,18.290140,16.493486,10471.541020,88439.60938,46.359188,265563.71880,2016.838379,0.000076,0.914638,3216.0,674.791667,674.791667
3,3,10.081685,53.529020,24.249276,23.645823,18.248226,11165.945310,76961.00000,0.000000,97921.56250,478.363281,0.000074,0.712983,1052.0,0.000000,0.000000
4,4,10.096292,48.847917,19.912116,14.026642,5.514785,6017.511230,30814.58984,0.000000,176265.68750,3338.221191,0.000047,1.000641,3240.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,477,9.979031,48.396836,27.221808,29.130899,12.865678,10829.854490,54732.38281,0.000000,198280.70310,3127.346436,0.000058,0.964609,2607.0,0.000000,0.000000
478,478,9.981599,53.529158,32.763475,33.719387,36.813637,9855.304688,83610.52344,0.000000,117941.50000,1480.759766,0.000088,1.000696,NaN,0.000000,0.000000
479,479,9.990569,53.507930,24.966811,17.919800,25.516184,9541.226562,76877.67188,0.000000,94117.64063,2670.007568,0.000082,0.485096,4338.0,0.000000,0.000000
480,480,9.997256,53.681993,39.256824,13.382681,7.228415,7302.048828,44593.98438,54.294807,171093.43750,2462.917969,0.000063,1.000694,4006.0,674.791667,674.791667


In [3]:
#remove unique idenifier & geodata
df = df.drop(['Unnamed: 0', 'Longitude', 'Latitude'], axis=1)
#replace NaN's with 0's
df = df.fillna(0)

#initialize response- and predictor variables
y = df['mean_value_NO2']
x = df.drop(['mean_value_NO2'], axis=1)
feature_list = list(x.columns)

## == CONSTRUCTING LightGBM == ##
# Import the model we are using

##STEP GENERATING CROSS VALIDATION (75% TRAINING OF MODEL; 
        ##25% TESTING OF MODEL;EVALUATING MODEL PERFORMANCE)

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=20)
#random state is needed to ensure that same results are generated each time.

## ENSURING THAT X- AND Y-TRAINING SET CONTAIN SAME AMOUNT OF ROWS. SAME FOR TESTING.
print('Training Features Shape:',X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Training Features Shape: (361, 12)
Training Labels Shape: (361,)
Testing Features Shape: (121, 12)
Testing Labels Shape: (121,)


In [4]:

## PARAMETERS FOR MODEL
model = lgb.LGBMRegressor(random_state=42)
model.fit(X_train, Y_train)

LGBMRegressor(random_state=42)

In [5]:
# ## PARAMETERS FOR MODEL
# model = lgb.LGBMRegressor(reg_alpha =2, reg_lambda = 0, max_depth = 5, learning_rate = 0.002, n_estimators =2000, random_state=42)
# model.fit(X_train, Y_train)

In [6]:
##DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES
expected_y  = Y_test
predicted_y = model.predict(X_test)

In [7]:
expected_y_tr  = Y_train
predicted_y_tr = model.predict(X_train)

In [8]:
## == EVALUATING PERFORMANCE OF LIGHTGBM MODEL == ##

##RMSE

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print('RMSE training: ',rmse(predicted_y_tr, expected_y_tr))
print('RMSE testing: ', rmse(predicted_y, expected_y))

## R2

from sklearn.metrics import r2_score
print('R2 score training: ', r2_score(expected_y_tr, predicted_y_tr))
print('R2 score testing: ', r2_score(expected_y, predicted_y))

##MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print('MAE training: ', mean_absolute_error(expected_y_tr, predicted_y_tr))
print('MAE testing: ', mean_absolute_error(expected_y, predicted_y))

RMSE training:  2.81478763743841
RMSE testing:  7.628827143988592
R2 score training:  0.950020347809771
R2 score testing:  0.6819455741514758
MAE training:  1.7822547276506615
MAE testing:  5.493023883745601


In [9]:
#HYPERPARAMETER TUNING
#VERIFYING CURRENT RANDOM FOREST HYPERPARAMETERS 


# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model.get_params())

Parameters currently in use:

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': 42,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': 'warn',
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}


In [10]:
#CREATE PARAMETER GRID

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [11]:
##RANDOM SEARCH TRAINING (https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

# Use the random grid to search for best hyperparameters
# First create the base model to tune

model = lgb.LGBMRegressor(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
model_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
model_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=2. Current value: min_data_in_leaf=2
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: bootstrap


RandomizedSearchCV(cv=3, estimator=LGBMRegressor(random_state=42), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [12]:
model_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': False}

In [13]:
model_hyper = lgb.LGBMRegressor(n_estimators=200, min_samples_split=5, 
                                 min_samples_leaf=4, max_features='auto',
                                max_depth=10, bootstrap=True)

# Train the model on training data
model_hyper.fit(X_train, Y_train)

LGBMRegressor(bootstrap=True, max_depth=10, max_features='auto',
              min_samples_leaf=4, min_samples_split=5, n_estimators=200)

In [14]:
##DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES
expected_y_H  = Y_test
predicted_y_H = model_hyper.predict(X_test)

expected_y_tr_H = Y_train
predicted_y_tr_H = model_hyper.predict(X_train)

In [15]:
##RMSE

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print('RMSE training dataset with hypertuning: ', rmse(predicted_y_tr_H, expected_y_tr_H))
print('RMSE testing dataset with hypertuning: ', rmse(predicted_y_H, expected_y_H))

RMSE training dataset with hypertuning:  0.08320350821675997
RMSE testing dataset with hypertuning:  8.189190420588128


In [16]:
##R2

from sklearn.metrics import r2_score
print('R2 score training dataset with hypertuning: ', r2_score(expected_y_tr_H, predicted_y_tr_H))
print('R2 score testing dataset with hypertuning: ', r2_score(expected_y_H, predicted_y_H))

R2 score training dataset with hypertuning:  0.9999563297944141
R2 score testing dataset with hypertuning:  0.633505186102966


In [17]:
##MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print('MAE training dataset with hypertuning: ', mean_absolute_error(expected_y_tr_H, predicted_y_tr_H))
print('MAE testing dataset with hypertuning: ', mean_absolute_error(expected_y_H, predicted_y_H))

MAE training dataset with hypertuning:  0.05304068860180971
MAE testing dataset with hypertuning:  5.681613298016954


In [18]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
model = lgb.LGBMRegressor(random_state=42)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [19]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train)
grid_search.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[LightGBM] [Warning] Unknown parameter: bootstrap
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=4. Current value: min_data_in_leaf=4
[LightGBM] [Warning] Unknown parameter: min_samples_split


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 2,
 'min_samples_leaf': 4,
 'min_samples_split': 8,
 'n_estimators': 100}

In [20]:
from sklearn.model_selection import GridSearchCV


# Create best grid model
model_grid = lgb.LGBMRegressor(random_state=42, bootstrap=True, max_depth=80, max_features=2, 
                         min_samples_leaf=3, min_samples_split=8, n_estimators=100)
model_grid.fit(X_train, Y_train)

LGBMRegressor(bootstrap=True, max_depth=80, max_features=2, min_samples_leaf=3,
              min_samples_split=8, random_state=42)

In [21]:
##DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TESTING DATASET)
expected_ygrid  = Y_test
predicted_ygrid = model_grid.predict(X_test) #Predicting the test set results using the random forest regressor model

In [22]:
##DEFINING ORIGINAL Y-VALUES AND PREDICTED Y-VALUES (FOR TRAINING DATASET)

expected_y_tr_grid = Y_train
predicted_y_tr_grid = model_grid.predict(X_train)
# print(expected_y_tr, predicted_y_tr)

In [23]:
##RMSE - TRAIN

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print("training rmse: ", rmse(predicted_y_tr_grid, expected_y_tr_grid))
print("testing rmse: ", rmse(predicted_ygrid, Y_test))

training rmse:  0.24104046526847886
testing rmse:  8.245471519179588


In [24]:
##R2

from sklearn.metrics import r2_score
print("R2 score training data: ", r2_score(expected_y_tr_grid, predicted_y_tr_grid))
print("R2 score testing data: ", r2_score(Y_test, predicted_ygrid))

R2 score training data:  0.9996334933376974
R2 score testing data:  0.628450324898779


In [25]:
##MEAN ABSOLUTE ERROR 
from sklearn.metrics import mean_absolute_error
print("MAE training data: ", mean_absolute_error(expected_y_tr_grid, predicted_y_tr_grid))
print("MAE testing data: ", mean_absolute_error(expected_ygrid, predicted_ygrid))

MAE training data:  0.18001229666706053
MAE testing data:  5.58636134407224
